**Step0. Getting a full list of today's matches recording their URLs**

In [31]:
import requests
from bs4 import BeautifulSoup
result = requests.get("http://www.tennisexplorer.com/matches/?type=all&year=2021&month=12&day=7")
src = result.content
soup = BeautifulSoup(src, 'lxml')
#print([tag.find("a")["href"] for tag in soup.select("td:has(a)")])
todays_list=[qwe.text.strip() for qwe in soup.select('a[href*="/match-detail/?id="]')]
links=[]
for q in soup.select('a[href*="/match-detail/?id="]'):
    links.append(q['href'])

todays_list=list(zip(todays_list, links))

todays_list_=[]
for idx, item in enumerate(todays_list):
    if todays_list[idx][0]=='info':
        #В todays_list_ забираем только вторую колонку изначального списка todays_list_
        todays_list_.append(todays_list[idx][1])
len(todays_list_)
#todays_list_

268

**Step1. Here should be a loop to scrape each match one by one. But I took 1st match for example and further processing**

In [32]:
#result = requests.get("http://www.tennisexplorer.com/match-detail/?id=2007904")
get_link= "http://www.tennisexplorer.com" + todays_list_[0]
#SAMUDIO get_link= "http://www.tennisexplorer.com/match-detail/?id=2007904"
#1. get_link= "https://www.tennisexplorer.com/match-detail/?id=2020226"
#get_link= "https://www.tennisexplorer.com/match-detail/?id=2008215"
result = requests.get(get_link)
src = result.content
soup = BeautifulSoup(src, 'lxml')

**Step2. Players' name and personal pages of each other**

In [33]:

import pandas as pd
import numpy as np
soup.select('th[class="plName"]')
#имена забрали со странички матча
lst=[qwe.text.strip() for qwe in soup.select('th[class="plName"]')]
np_lst = np.array(lst)

lst_=np.reshape(np_lst, (1, 2))
df = pd.DataFrame(lst_, columns=["p1", "p2"])
df

links=[]

req= soup.find_all("th", {"class":"plName"})
type(req)
links=[]
for w in req:
    #неправильно
    #links.append(w.get('href'))
        #надо так
        #links.append(w.find('a')['href'])
    #или так
    links.append(w.a.get('href'))
        #или так
        #rows = req.findAll("td")
        #linkelement = rows[linkpos]
        #a_element = linkelement.find('a')
        #a_element.get('href')
links

['/player/vecic', '/player/rumpf']

**Step3. Basic info about players**

In [35]:
import re
get_player= "http://www.tennisexplorer.com" + links[0]
result_ = requests.get(get_player)
src_ = result_.content
soup_ = BeautifulSoup(src_, 'lxml')
#columns = soup_.findAll('div', text = re.compile('Age'), attrs = {'class' : 'date'})
#[qwe.text.strip() for qwe in columns]
columns=soup_.findAll('div', {'class' : 'date'})
len([qwe.text.strip() for qwe in columns])
lst_p1=[qwe.text.strip() for qwe in soup_.select('div[class="date"]')]

get_player= "http://www.tennisexplorer.com" + links[1]
result_ = requests.get(get_player)
src_ = result_.content
soup_ = BeautifulSoup(src_, 'lxml')
lst_p2=[qwe.text.strip() for qwe in soup_.select('div[class="date"]')]
print(lst_p2)

['Country: Germany', 'Age: 19 (1. 1. 2003)', 'Sex: woman', 'Plays: right']


**Step4. Save first scraped info to our dashboard**

In [45]:
def podstava(new_col, like, list):
    for i in range(len(list)):
        if (like in list[i]):
            df[new_col]=list[i][list[i].find(':')+2:]
            break
        else:
            df[new_col]='-'

podstava('Country_p1', 'Countr', lst_p1)
podstava('Sgl_rank_p1', 'singles', lst_p1)
podstava('Age_p1', 'Age', lst_p1)
podstava('Dbl_rank_p1', 'doubles', lst_p1)
podstava('Hand_p1', 'Plays', lst_p1)

podstava('Country_p2', 'Countr', lst_p2)
podstava('Sgl_rank_p2', 'singles', lst_p2)
podstava('Age_p2', 'Age', lst_p2)
podstava('Dbl_rank_p2', 'doubles', lst_p2)
podstava('Hand_p2', 'Plays', lst_p2)
df

,p1,p2,Country_p1,Sgl_rank_p1,Age_p1,Dbl_rank_p1,Hand_p1,Country_p2,Sgl_rank_p2,Age_p2,Dbl_rank_p2,Hand_p2,W_p1,L_p1,W_p2,L_p2,coeff
0,Vecic Alexandra,Rumpf Paula,Germany,497. / 497.,20 (7. 1. 2002),1437. / 1425.,right,Germany,-,19 (1. 1. 2003),-,right,43,57,7,5,no coeff


**Step5. Total career win/lose results (Player1, Player2)**

In [37]:
print("TOTAL career (w/l):",end=' ')
wl_total=[qwe.text.strip() for qwe in soup.select('a[href$="?annual=all"]')]
print(wl_total)
#----2021 win/lose results (Player1, Player2) on different surfaces
print("2021 career (w/l):",end=' ')
wl_2021=[qwe.text.strip() for qwe in soup.select('a[href*="?annual=2021"]')]
print(wl_2021)

TOTAL career (w/l): ['43/57', '7/5']
2021 career (w/l): ['2021', '15/17', '0/3', '4/4', '2021', '7/4', '0/1']


**Step6. Put more scraped info to our dashboard**

In [38]:
def wl(i):
    df['W_p'+str(i+1)]=int(wl_total[i][:wl_total[i].find('/')])
    df['L_p'+str(i+1)]=int(wl_total[i][wl_total[i].find('/')+1:])
    
wl(0)
wl(1)
df

,p1,p2,Country_p1,Sgl_rank_p1,Age_p1,Dbl_rank_p1,Hand_p1,Country_p2,Sgl_rank_p2,Age_p2,Dbl_rank_p2,Hand_p2,W_p1,L_p1,W_p2,L_p2
0,Vecic Alexandra,Rumpf Paula,Germany,497. / 497.,20 (7. 1. 2002),1437. / 1425.,right,Germany,-,19 (1. 1. 2003),-,right,43,57,7,5


**Step7. Extracting match odds**

In [39]:
try:
    s= [qwe.text.strip() for qwe in soup.select_one('.average').findAll('div', {'class' : 'odds-in'})]
    df['coeff'] = s[0] + ' vs ' + s[1]
except:
    df['coeff'] = 'no coeff'
df

,p1,p2,Country_p1,Sgl_rank_p1,Age_p1,Dbl_rank_p1,Hand_p1,Country_p2,Sgl_rank_p2,Age_p2,Dbl_rank_p2,Hand_p2,W_p1,L_p1,W_p2,L_p2,coeff
0,Vecic Alexandra,Rumpf Paula,Germany,497. / 497.,20 (7. 1. 2002),1437. / 1425.,right,Germany,-,19 (1. 1. 2003),-,right,43,57,7,5,no coeff


**Step8. Detailed results of last 10matches**

In [40]:
#----Player1
pl1=soup.findAll('div', {'class' : 'half-l'})
for w in pl1:
    s=w.findAll('td', {'class' : 'score'})
score_p1=[qwe.text.strip() for qwe in s]
scores_p1=[]
for w in s:
    scores_p1.append(w.a.get('title'))

#---Player1's competitors
for w in pl1:
    s=w.findAll('td', {'class' : 'tl'})
participants_p1=[]
for w in s:
    if "/player/" in w.a.get('href'):
        links = w.find_all('a', href=True)
        participants_p1.append([qwe.text.strip() for qwe in links])
        ss=w.find_all(attrs={"class": "notU"} ,href=True)
        for i in ss:
            p1=i.attrs["href"]
p1=p1[p1.find('/',2)+1:-1]

        
#----Player2
pl2=soup.findAll('div', {'class' : 'half-r'})
for w in pl2:
    s=w.findAll('td', {'class' : 'score'})
score_p2=[qwe.text.strip() for qwe in s]
scores_p2=[]
for w in s:
    scores_p2.append(w.a.get('title'))


#---Player2's competitors
for w in pl2:
    s=w.findAll('td', {'class' : 'tl'})
participants_p2=[]
for w in s:
    if "/player/" in w.a.get('href'):
        links = w.find_all('a', href=True)
        participants_p2.append([qwe.text.strip() for qwe in links])
        ss=w.find_all(attrs={"class": "notU"} ,href=True)
        #---фамилии участников для сортировки
        for i in ss:
            p2=i.attrs["href"]
p2=p2[p2.find('/',2)+1:-1]

print(score_p1)
print(scores_p1)
print(participants_p1)
print(score_p2)
print(scores_p2)
print(participants_p2)



['1:0', '2:0', '2:0', '2:1', '2:0', '2:1', '2:0', '2:0', '2:1', '2:0']
['7-5, 0-0', '6-1, 6-0', '7-5, 6-2', '6-1, 3-6, 6-2', '6-4, 6-4', '5-7, 6-3, 10-6', '6-3, 7-6(6)', '6-0, 6-0', '6-4, 1-6, 6-3', '7-6(2), 6-0']
[['Gimbrere', 'Vecic'], ['Fruhvirtova', 'Vecic'], ['Vecic', 'Herdzelas'], ['Vecic', 'Kiick'], ['Vecic', 'Hibino'], ['Vecic', 'Rosca'], ['Vecic', 'Matoula'], ['Vecic', 'Imbo Nloga'], ['Aikawa', 'Vecic'], ['Vecic', 'Welker']]
['2:0', '2:0', '2:0', '2:1', '2:0', '1:0', '2:1', '2:1', '2:1', '2:0']
['6-2, 6-3', '7-5, 6-2', '7-6(6), 6-4', '2-6, 6-3, 11-9', '6-4, 7-6(5)', '5-2', '2-6, 7-6(4), 6-3', '1-6, 6-4, 10-6', '6-4, 5-7, 10-7', '6-1, 6-4']
[['Vecic', 'Rumpf'], ['Rumpf', 'Tilbuerger'], ['Lukacs', 'Rumpf'], ['Roemer', 'Rumpf'], ['Rumpf', 'Klasen'], ['Gettwart', 'Rumpf'], ['Rumpf', 'Hoedt'], ['Rumpf', 'Rioux'], ['Rumpf', 'Ruppert'], ['Rumpf', 'Rath']]


**Step9. Detailed results of last 10matches of Player1's competitors**

In [41]:
for w in pl1:
    s=w.findAll('td', {'class' : 'score'})

#---ИГРОК1. общая стата его соперников в 10 прошлых матчах
for w in pl1:
    s=w.findAll('td', {'class' : 'tl'})
links_p1=[]
for w in s:
    if "/player/" in w.a.get('href'):
        ss=w.find_all(attrs={"class": ""} ,href=True)
        for i in ss:
            links_p1.append(i.attrs["href"])
links_p1
    
#----Step10. Detailed results of last 10matches of Player2's competitors
for w in pl2:
    s=w.findAll('td', {'class' : 'tl'})
links_p2=[]
for w in s:
    if "/player/" in w.a.get('href'):
        ss=w.find_all(attrs={"class": ""} ,href=True)
        for i in ss:
            links_p2.append(i.attrs["href"])
links_p2

print(links_p1)
print(chr(13))
print(links_p2)
print(chr(13))


['/player/gimbrere/', '/player/fruhvirtova/', '/player/herdzelas/', '/player/kiick/', '/player/hibino/', '/player/rosca-adc63/', '/player/matoula-3cf1e/', '/player/imbo-nloga/', '/player/aikawa/', '/player/welker/']

['/player/vecic/', '/player/tilbuerger/', '/player/lukacs-bac4d/', '/player/roemer/', '/player/klasen/', '/player/gettwart/', '/player/hoedt/', '/player/rioux-b7f0f/', '/player/ruppert-32ef3/', '/player/rath-69c23/']



**Step10. Buildind 2nd block of dashboard (Player1's part)**

In [42]:
dff=pd.DataFrame(participants_p1,columns =['p1','p2'])
dff['sets']=score_p1
dff['games']=scores_p1
dff['alignment by']=p1
dff[['set_p1','set_p2']] = dff["sets"].str.split(":", n = 1, expand = True)
dff[['s1p1_g','s1p2_g','s2p1_g','s2p2_g','s3p1_g','s3p2_g']]  = dff["games"].str.split("/|;|_|%|-|,",expand=True)
dff=dff.fillna(0)
    #dff.apply(lambda col: col.str.contains("\(", na=False), axis=1)
def mochi_skobki(col):
    list=dff[col]
    for i in range(len(list)):
        if str(list[i]).find('(')==True:
            list[i]=str(list[i])[:str(list[i]).find('(')]
    dff[col]=list
mochi_skobki('s1p1_g')
mochi_skobki('s1p2_g')
mochi_skobki('s2p1_g')
mochi_skobki('s2p2_g')
mochi_skobki('s3p1_g')
mochi_skobki('s3p2_g')

for index in range(dff.shape[0]):
    if dff.loc[index, 'p1'].lower() not in dff.loc[index, 'alignment by']:
        dff.loc[index,['p1','p2']] = dff.loc[index,['p2','p1']].values
        dff.loc[index,['set_p1','set_p2']] = dff.loc[index,['set_p2','set_p1']].values
        dff.loc[index,['s1p1_g','s1p2_g']] = dff.loc[index,['s1p2_g','s1p1_g']].values
        dff.loc[index,['s2p1_g','s2p2_g']] = dff.loc[index,['s2p2_g','s2p1_g']].values
        dff.loc[index,['s3p1_g','s3p2_g']] = dff.loc[index,['s3p2_g','s3p1_g']].values
dff

rival_p1_gamerk=[]
rival_p1_rating=[]

for idx in range(len(links_p1)):

    get_link= "http://www.tennisexplorer.com" + links_p1[idx]
    result = requests.get(get_link)
    src = result.content
    soup_p1 = BeautifulSoup(src, 'lxml')

    #----------------------rival_p1_gamerk
    rival_p1_gamerk.append([qwe.text.strip() for qwe in soup_p1.select('a[href$="?annual=all"]')])
    #----------------------rival_p1_rating

    #columns=soup_p1.findAll('div', {'class' : 'date'})
    #len([qwe.text.strip() for qwe in columns])
    rival_p1=[qwe.text.strip() for qwe in soup_p1.select('div[class="date"]')]
    for i in range(len(rival_p1)):
        fl=False
        if ('singles' in rival_p1[i]):
            fl=True
            rival_p1_rating.append(rival_p1[i][rival_p1[i].find(':')+2:])
            break
    if fl==False:
        rival_p1_rating.append('-')

dff['rival_p1_gamerk']=rival_p1_gamerk
dff['rival_p1_rating']=rival_p1_rating

**Step11. Buildind 3rd block of dashboard (Player2's part)**

In [43]:
d=pd.DataFrame(participants_p2,columns =['p1','p2'])
d['sets']=score_p2
d['games']=scores_p2
d['alignment by']=p2
d[['set_p1','set_p2']] = d["sets"].str.split(":", n = 1, expand = True)
d[['s1p1_g','s1p2_g','s2p1_g','s2p2_g','s3p1_g','s3p2_g']]  = d["games"].str.split("/|;|_|%|-|,",expand=True)
d=d.fillna(0)
    #d.apply(lambda col: col.str.contains("\(", na=False), axis=1)
def mochi_skobki(col):
    list=d[col]
    for i in range(len(list)):
        if str(list[i]).find('(')==True:
            list[i]=str(list[i])[:str(list[i]).find('(')]
    d[col]=list
mochi_skobki('s1p1_g')
mochi_skobki('s1p2_g')
mochi_skobki('s2p1_g')
mochi_skobki('s2p2_g')
mochi_skobki('s3p1_g')
mochi_skobki('s3p2_g')

for index in range(d.shape[0]):
    if d.loc[index, 'p1'].lower() not in d.loc[index, 'alignment by']:
        d.loc[index,['p1','p2']] = d.loc[index,['p2','p1']].values
        d.loc[index,['set_p1','set_p2']] = d.loc[index,['set_p2','set_p1']].values
        d.loc[index,['s1p1_g','s1p2_g']] = d.loc[index,['s1p2_g','s1p1_g']].values
        d.loc[index,['s2p1_g','s2p2_g']] = d.loc[index,['s2p2_g','s2p1_g']].values
        d.loc[index,['s3p1_g','s3p2_g']] = d.loc[index,['s3p2_g','s3p1_g']].values
d

rival_p1_gamerk=[]
rival_p1_rating=[]

for idx in range(len(links_p1)):

    get_link= "http://www.tennisexplorer.com" + links_p2[idx]
    result = requests.get(get_link)
    src = result.content
    soup_p1 = BeautifulSoup(src, 'lxml')

    #----------------------rival_p1_gamerk
    rival_p1_gamerk.append([qwe.text.strip() for qwe in soup_p1.select('a[href$="?annual=all"]')])
    #----------------------rival_p1_rating

    #columns=soup_p1.findAll('div', {'class' : 'date'})
    #len([qwe.text.strip() for qwe in columns])
    rival_p1=[qwe.text.strip() for qwe in soup_p1.select('div[class="date"]')]
    for i in range(len(rival_p1)):
        fl=False
        if ('singles' in rival_p1[i]):
            fl=True
            rival_p1_rating.append(rival_p1[i][rival_p1[i].find(':')+2:])
            break
    if fl==False:
        rival_p1_rating.append('-')

d['rival_p1_gamerk']=rival_p1_gamerk
d['rival_p1_rating']=rival_p1_rating

**Step12. Getting an entire Dashboard for further convenient and smart analysis**

In [44]:
display(df)
display(dff)
display(d)

,p1,p2,Country_p1,Sgl_rank_p1,Age_p1,Dbl_rank_p1,Hand_p1,Country_p2,Sgl_rank_p2,Age_p2,Dbl_rank_p2,Hand_p2,W_p1,L_p1,W_p2,L_p2,coeff
0,Vecic Alexandra,Rumpf Paula,Germany,497. / 497.,20 (7. 1. 2002),1437. / 1425.,right,Germany,-,19 (1. 1. 2003),-,right,43,57,7,5,no coeff


,p1,p2,sets,games,alignment by,set_p1,set_p2,s1p1_g,s1p2_g,s2p1_g,s2p2_g,s3p1_g,s3p2_g,rival_p1_gamerk,rival_p1_rating
0,Vecic,Gimbrere,1:0,"7-5, 0-0",vecic,0,1,5,7,0,0,0,0,[56/44],670. / 670.
1,Vecic,Fruhvirtova,2:0,"6-1, 6-0",vecic,0,2,1,6,0,6,0,0,[55/35],281. / 276.
2,Vecic,Herdzelas,2:0,"7-5, 6-2",vecic,2,0,7,5,6,2,0,0,[304/184],206. / 204.
3,Vecic,Kiick,2:1,"6-1, 3-6, 6-2",vecic,2,1,6,1,3,6,6,2,[235/165],262. / 126.
4,Vecic,Hibino,2:0,"6-4, 6-4",vecic,2,0,6,4,6,4,0,0,[284/231],161. / 56.
5,Vecic,Rosca,2:1,"5-7, 6-3, 10-6",vecic,2,1,5,7,6,3,10,6,[53/35],835. / 835.
6,Vecic,Matoula,2:0,"6-3, 7-6(6)",vecic,2,0,6,3,7,6,0,0,[77/50],792. / 790.
7,Vecic,Imbo Nloga,2:0,"6-0, 6-0",vecic,2,0,6,0,6,0,0,0,[1/13],-
8,Vecic,Aikawa,2:1,"6-4, 1-6, 6-3",vecic,1,2,4,6,6,1,3,6,[106/92],946. / 817.
9,Vecic,Welker,2:0,"7-6(2), 6-0",vecic,2,0,7,6,6,0,0,0,[48/54],965. / 930.


,p1,p2,sets,games,alignment by,set_p1,set_p2,s1p1_g,s1p2_g,s2p1_g,s2p2_g,s3p1_g,s3p2_g,rival_p1_gamerk,rival_p1_rating
0,Rumpf,Vecic,2:0,"6-2, 6-3",rumpf,0,2,2,6,3,6,0,0,[43/57],497. / 497.
1,Rumpf,Tilbuerger,2:0,"7-5, 6-2",rumpf,2,0,7,5,6,2,0,0,[96/86],771. / 742.
2,Rumpf,Lukacs,2:0,"7-6(6), 6-4",rumpf,0,2,6,7,4,6,0,0,[388/296],530. / 369.
3,Rumpf,Roemer,2:1,"2-6, 6-3, 11-9",rumpf,1,2,6,2,3,6,9,11,[184/151],961. / 539.
4,Rumpf,Klasen,2:0,"6-4, 7-6(5)",rumpf,2,0,6,4,7,6,0,0,[260/214],544. / 426.
5,Rumpf,Gettwart,1:0,5-2,rumpf,0,1,2,5,0,0,0,0,[47/44],934. / 934.
6,Rumpf,Hoedt,2:1,"2-6, 7-6(4), 6-3",rumpf,2,1,2,6,7,6,6,3,[100/86],600. / 600.
7,Rumpf,Rioux,2:1,"1-6, 6-4, 10-6",rumpf,2,1,1,6,6,4,10,6,[64/51],860. / 561.
8,Rumpf,Ruppert,2:1,"6-4, 5-7, 10-7",rumpf,2,1,6,4,5,7,10,7,[84/63],896. / 817.
9,Rumpf,Rath,2:0,"6-1, 6-4",rumpf,2,0,6,1,6,4,0,0,[15/28],-
